# IPython Blockfolio

This is a simple "Blockfolio" using Python and the [CoinMarketCap API](https://coinmarketcap.com/api/).

It will show the net worth in USD of your Cryptocurrency investments as well as the relative share
of coins you own in comparison to the total coin supply of the corresponding digital asset.

First, the current exchanges rates are loaded, then combined with your investments, and finally displayed using Plotly.

This notebook can be found on [my github profile](https://github.com/SmokinCaterpillar/blockfolio).

Imports
--------

In [34]:
import pandas as pd

import plotly
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)

## Loading Data from CoinMarkteCap

In [35]:
limit = 200  # only load the top 200 currencies, if you have invested in smaller ones increase this limit
coin_market_cap_api = 'https://api.coinmarketcap.com/v1/ticker/?limit={}'.format(limit)
coin_market_cap_api

'https://api.coinmarketcap.com/v1/ticker/?limit=200'

In [36]:
# load the data using pandas and keep track of the time
now = pd.datetime.now()
market_data = pd.read_json(coin_market_cap_api)
market_data.head(10)

,24h_volume_usd,available_supply,id,last_updated,market_cap_usd,max_supply,name,percent_change_1h,percent_change_24h,percent_change_7d,price_btc,price_usd,rank,symbol,total_supply
0,5483020000,16946362,bitcoin,1522344268,127525610640,2.100000e+07,Bitcoin,-0.45,-5.19,-13.26,1.000000,7525.250000,1,BTC,16946362
1,1768970000,98491406,ethereum,1522344253,40879350517,NaN,Ethereum,0.10,-7.68,-22.41,0.055434,415.055000,2,ETH,98491406
2,371191000,39094227299,ripple,1522344241,21310341489,1.000000e+11,Ripple,-0.61,-5.42,-16.75,0.000073,0.545102,3,XRP,99992434971
3,405845000,17044388,bitcoin-cash,1522344252,12877137023,2.100000e+07,Bitcoin Cash,-0.96,-12.61,-24.97,0.100903,755.506000,4,BCH,17044388
4,393255000,55845045,litecoin,1522344241,6834875007,8.400000e+07,Litecoin,-0.64,-7.98,-25.26,0.016346,122.390000,5,LTC,55845045
5,543223000,758247977,eos,1522344253,4755048889,1.000000e+09,EOS,-0.78,-0.31,-7.86,0.000838,6.271100,6,EOS,900000000
6,89176200,25927070538,cardano,1522344256,4020225631,4.500000e+10,Cardano,-0.62,-2.81,-22.29,0.000021,0.155059,7,ADA,31112483745
7,40536400,18550593129,stellar,1522344243,3755344972,NaN,Stellar,-0.81,-7.32,-14.95,0.000027,0.202438,8,XLM,103807834721
8,88059600,65000000,neo,1522344248,3525333500,1.000000e+08,NEO,-0.31,-5.14,-21.78,0.007244,54.235900,9,NEO,100000000
9,485474000,65748192475,tron,1522344254,3224961991,NaN,TRON,-1.28,9.79,36.32,0.000007,0.049050,10,TRX,100000000000


## Your Blockfolio

Below you can fill in the types and amount of coins you have into the Python dictionary
with the format `'SYMBOL': value`.

In [37]:
# enter your coins below:

block_folio = {
    'BTC': 0.31337,
    'BCH': 0.01337,
    'ETH': 3,
    'LTC': 1.5,
    'MIOTA': 777,
    'XRP': 13,
    'DASH': 0.1, 
    'XMR': 1, 
    'LSK': 12, 
    'OMG': 1.1, 
    'XRB': 42, 
    'PIVX': 123, 
    'ARK': 22, 
    'NEO': 7, 
    'GAS': 0.01,
    'STEEM': 499,
}

In [38]:
# ----- OPTIONAL: uncomment for keeping track of multiple wallets ----- #

# if you have some coins spread out over multiple wallets that might change value independently
# you can enter individual balances here:

#btc = { 'binance': 0.002, 'gdax': 0.05, 'cold_storage': 0.1}
#eth = { 'binance': 0.05, 'bittrex': 1}

# and then use the sum of your holdings for the coin's block_folio value:

#block_folio['BTC'] = sum(btc.values())
#block_folio['ETH'] = sum(eth.values())

In [39]:
# display your blockfolio
block_folio = pd.DataFrame(list(block_folio.items()), columns=['symbol', 'amount'])
block_folio.head(len(block_folio))

,symbol,amount
0,DASH,0.10000
1,BCH,0.01337
2,NEO,7.00000
3,PIVX,123.00000
4,GAS,0.01000
5,LSK,12.00000
6,XRB,42.00000
7,LTC,1.50000
8,MIOTA,777.00000
9,BTC,0.31337


### Combining the Blockfolio and the API Data

In [40]:
# merge the API and blockfolio data and sort by investment value
merged_data = block_folio.merge(market_data, how='left')
merged_data['value_usd'] = merged_data.amount * merged_data.price_usd
merged_data['coinshare'] = merged_data.amount / merged_data.available_supply
merged_data = merged_data.sort_values('value_usd', ascending=False)
merged_data.head()

,symbol,amount,24h_volume_usd,available_supply,id,last_updated,market_cap_usd,max_supply,name,percent_change_1h,percent_change_24h,percent_change_7d,price_btc,price_usd,rank,total_supply,value_usd,coinshare
9,BTC,0.31337,5483020000,16946362,bitcoin,1522344268,1.275256e+11,21000000,Bitcoin,-0.45,-5.19,-13.26,1.000000,7525.25000,1,16946362,2358.187592,1.849187e-08
11,ETH,3.00000,1768970000,98491406,ethereum,1522344253,4.087935e+10,NaN,Ethereum,0.10,-7.68,-22.41,0.055434,415.05500,2,98491406,1245.165000,3.045951e-08
8,MIOTA,777.00000,33912400,2779530283,iota,1522344251,3.128334e+09,2779530283,IOTA,-0.64,-6.17,-14.20,0.000150,1.12549,11,2779530283,874.505730,2.795436e-07
13,STEEM,499.00000,3353340,255563840,steem,1522344246,4.432346e+08,NaN,Steem,-0.13,-3.88,-16.87,0.000232,1.73434,31,272537934,865.435660,1.952545e-06
3,PIVX,123.00000,4932590,55853638,pivx,1522344246,2.188848e+08,NaN,PIVX,-2.08,-10.82,1.50,0.000523,3.91890,58,55853638,482.024700,2.202184e-06


### Your Blockfolio in numbers and graphs!


In [41]:
networth = merged_data.value_usd.sum()

print('Your blockfolio is currently (i.e {}) worth {:.2f} USD!'.format(now.strftime('%Y-%m-%d %I:%M %p'), networth))

Your blockfolio is currently (i.e 2018-03-30 12:29 AM) worth 6786.07 USD!


#### Net Worth

Let's plot the worth of each asset in descending order:

In [42]:
marker = dict(color='rgb(158,202,225)',
              line=dict(color='rgb(8,48,107)', width=1.5))

layout = go.Layout(title='Blockfolio Networth in USD',
                   xaxis=dict(title='Cryptocurrency'),
                   yaxis=dict(title='USD'))
              
value_chart=go.Bar(x=merged_data.symbol, y=merged_data.value_usd, marker=marker)

fig = go.Figure(data=[value_chart], layout=layout)
py.iplot(fig)

#### Fraction of total Supply
Finally, that looks at the relative sizes of your investment. 
This gives you an idea or visualizes in which cryptocurrency you believe in the most. 
The graph below shows you fractional share of the the current crypto's total supply.

In [43]:
marker = dict(color='rgb(258,202,125)', line=dict(color='rgb(8,48,107)', width=1.5,))

layout = go.Layout(title='Relative Blockfolio Size',
                   xaxis=dict(title='Cryptocurrency'),
                   yaxis=dict(title='Fraction of total Supply'))

share_chart=go.Bar(x=merged_data.symbol, y=merged_data.coinshare, marker=marker)

fig = go.Figure(data=[share_chart], layout=layout)
py.iplot(fig)

## Your fiat investments

To keep track of how much fiat money (here in USD) was invested.

In [44]:
fiat_USD = {
    '2013-12-01': 100.5,
    '2017-10-15': 1000.10,
    '2018-01-31': 500.30,
}

investment = sum(fiat_USD.values())

print('By {} you invested {:.2f} USD!'.format(now.strftime('%Y-%m-%d %I:%M %p'), investment))

By 2018-03-30 12:29 AM you invested 1600.90 USD!


## Your current gains/losses

How much did your blockfolio increase or decrease in value?

In [45]:
capital_gains = networth - investment

print('At {} your crypto investment nets USD {}'.format(now.strftime('%Y-%m-%d %I:%M %p'), capital_gains))

At 2018-03-30 12:29 AM your crypto investment nets USD 5185.16663972
